In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

# Lecture 01: Simple Financial Assistant

This notebook demonstrates a basic financial assistant agent that uses web search to answer financial questions.

## How the Agent Works: ReAct Pattern

The agent follows the **ReAct** (Reasoning + Acting) pattern, which combines reasoning and acting in an iterative loop:

```mermaid
graph TD
    Start([👤 User Question ]) --> Reason[🤖 Agent: Analyze & Reason<br/>- Understand question<br/>- Plan approach<br/>- Decide on tools]
    Reason --> Decision{Need<br/>Tools?}
    
    Decision -->|No| Response[📝 Generate<br/>Final Response]
    Decision -->|Yes| CallTool[🔧 Call Tool<br/>Web Search]
    
    CallTool --> Results[📊 Receive<br/>Tool Results]
    Results --> React[🤖 Agent: React<br/>- Process results<br/>- Integrate information<br/>- Decide next step]
    
    React --> Decision
    
    Response --> End([✅ Return Answer<br/>to User])
    
    style Start fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style Reason fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Decision fill:#f3e5f5,stroke:#4a148c,stroke-width:2px
    style CallTool fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style Results fill:#e8f5e9,stroke:#1b5e20,stroke-width:2px
    style React fill:#fff9c4,stroke:#f57f17,stroke-width:2px
    style Response fill:#e0f2f1,stroke:#004d40,stroke-width:2px
    style End fill:#e1f5ff,stroke:#01579b,stroke-width:3px
```

**Key Points:**
- **Reasoning**: The agent analyzes the question and decides what actions to take
- **Acting**: The agent calls tools (like web search) to gather information
- **Iteration**: This process repeats until the agent has enough information (shown by the loop back from React to Decision)
- **Reflection**: After each tool call, the agent reflects on the results and decides next steps


In [ ]:
# Setup: Import necessary modules
from IPython.display import Markdown, display
from pydantic_ai import Agent
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging, setup_logfire

# Initialize logging
setup_logging()

# Initialize Logfire instrumentation (for observability)
setup_logfire(
    service_name="financial-assistant-lecture01",
    start_message="🚀 Lecture 01 - Financial Assistant Notebook Started",
)

In [ ]:
# Create the financial assistant agent
financial_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    system_prompt="""
You are a knowledgeable financial assistant. Your role is to help users with financial questions,
investment advice, market analysis, and general financial guidance.

When answering questions:
1. Use the web_search tool to get current, up-to-date financial information when relevant
2. Combine web search results with your financial knowledge
3. Provide clear, actionable advice
4. Always mention when information is based on web search vs. general knowledge
5. Include appropriate disclaimers for investment advice
6. Be helpful but remind users to consult with financial professionals for major decisions

Keep your responses informative but concise, and always prioritize accuracy and user safety.
""",
)

In [ ]:
# Ask a question
question = "What is the current price of Bitcoin?"
result = await financial_agent.run(question)

# Display with nice Markdown formatting
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
display(Markdown(result.output))

In [ ]:
# Try another question
question = "Find 3 trending stocks and for each one provide an investor's report"
result = await financial_agent.run(question)

# Display with nice Markdown formatting
display(Markdown(f"**Question:** {question}"))
display(Markdown("---"))
display(Markdown(result.output))